## Code from last lecture

In [4]:
import gzip
import itertools

In [5]:
def loadFasta(filename):
    """ Parses a classically formatted and possibly 
        compressed FASTA file into a list of headers 
        and fragment sequences for each sequence contained"""
    if (filename.endswith(".gz")):
        fp = gzip.open(filename, 'r')
        data = fp.read().decode("utf-8")
    else:
        fp = open(filename, 'r')
        data = fp.read()
    # split at headers
    data = data.split('>')
    fp.close()
    # ignore whatever appears before the 1st header
    data.pop(0)     
    headers = []
    sequences = []
    for sequence in data:
        lines = sequence.split('\n')
        headers.append(lines.pop(0))
        # add an extra "+" to make string "1-referenced"
        sequences.append('+' + ''.join(lines))
    return (headers, sequences)

In [11]:
import glob

for filename in sorted(glob.glob("../data/HumanChr*")):
    print(filename, len(header), "sequences")
    header, seq = loadFasta(filename)
    for i in range(len(header)):
        print(header[i])
        print(len(seq[i])-1, "bases", seq[i][:30], "...", seq[i][-30:])

../data/HumanChr01.fasta.gz 1 sequences
CM000663.2 Homo sapiens chromosome 1, GRCh38 reference primary assembly
248956422 bases +NNNNNNNNNNNNNNNNNNNNNNNNNNNNN ... NNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
../data/HumanChr02.fasta.gz 1 sequences
CM000664.2 Homo sapiens chromosome 2, GRCh38 reference primary assembly
242193529 bases +NNNNNNNNNNNNNNNNNNNNNNNNNNNNN ... NNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
../data/HumanChr03.fasta.gz 1 sequences
CM000665.2 Homo sapiens chromosome 3, GRCh38 reference primary assembly
198295559 bases +NNNNNNNNNNNNNNNNNNNNNNNNNNNNN ... NNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
../data/HumanChr04.fasta.gz 1 sequences
CM000666.2 Homo sapiens chromosome 4, GRCh38 reference primary assembly
190214555 bases +NNNNNNNNNNNNNNNNNNNNNNNNNNNNN ... NNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
../data/HumanChr05.fasta.gz 1 sequences
CM000667.2 Homo sapiens chromosome 5, GRCh38 reference primary assembly
181538259 bases +NNNNNNNNNNNNNNNNNNNNNNNNNNNNN ... NNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
../data/HumanChr06.fasta.gz 1 

## Output each chromosome as a simple string to speed up future processing

In [ ]:
header, seq = loadFasta("data/GCA_000001405.15_GRCh38_genomic.fna.gz")
print(len(header), "sequences")
for i in range(len(header)):
    if header[i].startswith("CM") or header[i].startswith("J0"):
        start = header[i].find('chromosome ')
        chromo = header[i][start+11:header[i].find(',')] if (start >= 0) else "MT"
        with open("data/Chr%s.seq" % chromo, 'w') as fp:
            fp.write(seq[i])            

## A function to construct the reverse-complement version of a DNA sequence

In [26]:
def revComp(dnaSeq):
    return ''.join([{'A':'T','C':'G','G':'C','T':'A'}[base] for base in reversed(dnaSeq)])

In [44]:
foo = {"A":"0", "C":"1", "G":"2", "T":"3"}
baseList = []
for base in reversed("GAGACAT"):
    baseList.append(foo[base])
print(''.join(baseList))

3010202


In [45]:
print(revComp("GAGACAT"))
print(revComp("ATGTCTC"))

ATGTCTC
GAGACAT


In [28]:
test = "cGTGATATTCCTTTTAAGAGGGAAGGgaaagaacagaaccaaaccaaaccaaaccacagataaggtaaaagaaaggttgatgaggtggagtccttcgacc".upper()
print revComp(test)

GGTCGAAGGACTCCACCTCATCAACCTTTCTTTTACCTTATCTGTGGTTTGGTTTGGTTTGGTTCTGTTCTTTCCCTTCCCTCTTAAAAGGAATATCACG


## Find the frequency of all k-mers in the genome

# Warning: This next cell takes a while...

In [ ]:
import time

DATA = "../HumanGenome/"  # change to match the directory where your chromosome sequences are.
chromo = [str(i) for i in xrange(1,23)] + ['X', 'Y', 'MT']

kmerCount = {}
K = 11
L = 0
for contig in chromo:
    tick = time.time()
    with open(DATA+"Chr%s.seq" % contig, 'r') as fp:
        seq = fp.read()
    for i in xrange(1,len(seq)-K+1):
        kmer = seq[i:i+K]
        for base in "RYSWKMBDHVN":
            if (base in kmer):
                break
        else:
            kmerCount[kmer] = kmerCount.get(kmer,0) + 1
            kmer = revComp(kmer)
            kmerCount[kmer] = kmerCount.get(kmer,0) + 1
    tock = time.time()
    print(contig, len(seq)-1, len(kmerCount), "%6.2f secs" % (tock - tick))
    tick = tock
    L += len(seq) - 1
    break
print(L, len(kmerCount))

In [ ]:
print(4**K - len(kmerCount))

## The code belows prints out all of the missing k-mers, if you're interested

In [20]:
missing = []
for pattern in itertools.product("ACGT", repeat=K):
    kmer = ''.join(pattern)
    if kmer not in kmerCount:
        missing.append(kmer)
print(len(missing))

167156


## Plot the kmer distribution

In [ ]:
import matplotlib
import matplotlib.pyplot as plot
%matplotlib inline

# Compute a histogram of kmer-counts (i.e. how many kmers appear 1 time, 2 times, 3 times ...)
maxcount = 500
hist = [0 for i in range(maxcount)]
for kmer in kmerCount:
    count = kmerCount[kmer]
    if (count < maxcount):
        hist[count] += 1

fig = plot.figure(figsize=(10,6))
plot.plot([i for i in range(maxcount)], hist)
plot.show()

In [29]:
contig = '1'
with open(DATA+"Chr%s.seq" % contig, 'r') as fp:
    seq = fp.read()
print(len(seq))

248956423


96166571
